In [ ]:
!uv pip install /kaggle/input/packages/peft-0.16.0-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/qwen_vl_utils-0.0.11-py3-none-any.whl --no-deps
# !uv pip install /kaggle/input/packages/transformers-4.53.3-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/datasets-3.6.0-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/accelerate-1.7.0-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl --no-deps
!uv pip install /kaggle/input/packages/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
# !pip install /kaggle/input/packages/torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl --no-deps
!uv pip install /kaggle/input/packages/torchvision-0.22.0-cp310-cp310-manylinux_2_28_x86_64.whl --no-deps
!uv pip install /kaggle/input/packages/wandb-0.19.11-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
!uv pip install /kaggle/input/packages/huggingface_hub-0.31.2-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/hf_xet-1.1.1-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
# !uv pip install /kaggle/input/packages/colpali_engine-0.3.12-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/docker_pycreds-0.4.0-py2.py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/decord-0.6.0-py3-none-manylinux2010_x86_64.whl --no-deps
!uv pip install /kaggle/input/packages/ms_swift-3.6.4-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/modelscope-1.28.1-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/deepspeed-0.16.5-py3-none-any.whl --no-deps
!uv pip install /kaggle/input/packages/hjson-3.1.0-py3-none-any.whl --no-deps

In [ ]:
!uv pip install --upgrade pandas==2.2.3
!uv pip install --upgrade --force-reinstall scikit-learn
!uv pip install --upgrade torchvision --no-deps
!uv pip install --upgrade transformers==4.51.3
!uv pip install sentence-transformers
!uv pip install --upgrade numpy==1.26.4
!uv pip install lmdeploy
!uv pip install qwen_vl_utils

In [ ]:
!lmdeploy serve api_server Qwen/Qwen2.5-VL-3B-Instruct --server-port 36365 --tp 1 --backend pytorch

In [ ]:
!rm -rf /kaggle/working/results.jsonl

In [1]:
# Cài đặt LMDeploy
# !pip install lmdeploy[all]
!rm -rf /kaggle/working/results.
import os 
import json
from PIL import Image
from lmdeploy import pipeline, TurbomindEngineConfig
from tqdm import tqdm
import time

# Khởi tạo pipeline
engine_config = TurbomindEngineConfig(session_len=1024, max_batch_size=1)
pipe = pipeline('Qwen/Qwen2.5-VL-3B-Instruct', backend_config=engine_config)

def make_inference_direct(prompt, image_path, pipe):
    """Direct inference using LMDeploy pipeline"""
    try:
        # Load image
        # image = Image.open(image_path).convert('RGB')
        
        # Create conversation
        messages = [
            {
                'role': 'user', 
                'content': [
                    {'type': 'text', 'text': prompt},
                    {'type': 'image_url', 'image_url': {"url" : image_path}}
                ]
            }
        ]
        
        # Generate response
        response = pipe(messages, max_new_tokens=20, temperature=0.01)
        return response.text.strip()
        
    except Exception as e:
        print(f"Direct inference error: {e}")
        return "ERROR"

# Load dataset
with open("/kaggle/input/vtqa-dataset/annotations/val.json", "r") as f:
    vtqa_dataset = json.load(f)

print(f"Loaded {len(vtqa_dataset)} samples")

# Main inference loop
output_file = "results.jsonl"
with open(output_file, "w", encoding="utf-8") as f_out:
    for i, item in tqdm(enumerate(vtqa_dataset), total=len(vtqa_dataset)):
        try:
            question = item['question']['raw']['en']
            context = item['context']['raw']['en']
            answer = item['answers'][0]['answer']['en']
            image_name = item['image_name']['image']
            image_path = os.path.join("/kaggle/input/vtqa-dataset/image/val", image_name)
            
            # Kiểm tra file ảnh tồn tại
            if not os.path.exists(image_path):
                print(f"Image not found: {image_path}")
                prediction = "ERROR: Image not found"
            else:
                # Tạo prompt dựa vào answer_type
                if item['answers'][0]['answer_type'] == "YN":
                    prompt = f"Given this context information: {context} \n\nQuestion: {question}\n\nAnswer with 'Yes' or 'No' only. If the entity being asked about appears in both the question and the context. Analyze first before answering the question."
                else:
                    prompt = f"Given this context information: {context}.\n\nQuestion: {question}\n\nAnswer as briefly as possible. If the entity being asked about appears in both the question and the context. Analyze first before answering the question."
                prompt += "Do not response with 'The question is not answerable' but you have to search more in the context"
                prediction = make_inference_direct(prompt, image_path, pipe)
            
            result_item = {
                'question': question,
                'pred': prediction,
                'gt': answer,
                'image_name': image_name,
                'answer_type': item['answers'][0]['answer_type']
            }
            
            # Ghi vào file
            f_out.write(json.dumps(result_item, ensure_ascii=False) + "\n")
            f_out.flush()
            
            # Log progress
            if i % 10 == 0 or i < 5:
                print(f"\nSample {i+1}/{len(vtqa_dataset)}:")
                print(f"Question: {question}")
                print(f"Predicted: {prediction}")
                print(f"Ground Truth: {answer}")
                print(f"Answer Type: {item['answers'][0]['answer_type']}")
                print("-" * 50)
            
        except Exception as e:
            print(f"Error processing sample {i}: {e}")
            result_item = {
                'question': question if 'question' in locals() else "ERROR",
                'pred': "ERROR",
                'gt': answer if 'answer' in locals() else "ERROR",
                'image_name': image_name if 'image_name' in locals() else "ERROR",
                'error': str(e)
            }
            f_out.write(json.dumps(result_item, ensure_ascii=False) + "\n")
            f_out.flush()

print(f"✅ Completed inference on {len(vtqa_dataset)} samples")
print(f"📄 Results saved to {output_file}")

# Đọc và hiển thị một số kết quả mẫu
print("\n📊 Sample results:")
with open(output_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 5:  # Chỉ hiển thị 5 kết quả đầu
            break
        result = json.loads(line)
        print(f"{i+1}. Q: {result['question']}")
        print(f"   Pred: {result['pred']}")
        print(f"   GT: {result['gt']}")
        print()

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

2025-09-18 05:21:04.991064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758172865.013313     904 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758172865.020084     904 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


2025-09-18 05:21:14,710 - lmdeploy - WARNING - turbomind.py:280 - get 327 model params


[TM][WARNING] [LlamaTritonModel] `max_context_token_num` is not set, default to 1024.
[TM][WARNING] [SegMgr] prefix caching is disabled                           


Loaded 1245 samples


...


Sample 1231/1245:
Question: In which Olympics did the sport first appear in the picture?
Predicted: 1984 Los Angeles Olympics
Ground Truth: The first modern Olympic Games were held in Athens in 1896
Answer Type: E
--------------------------------------------------


[TM][WARNING] [ProcessInferRequests] [1231] total sequence length (663 + 512) exceeds `session_len` (1024), `max_new_tokens` is truncated to 360
100%|█████████▉| 1241/1245 [06:33<00:01,  3.02it/s]


Sample 1241/1245:
Question: Is the surface of the ball used in the sport in the picture smooth?
Predicted: Yes
Ground Truth: Yes
Answer Type: YN
--------------------------------------------------


[TM][WARNING] [ProcessInferRequests] [1241] total sequence length (748 + 512) exceeds `session_len` (1024), `max_new_tokens` is truncated to 275
100%|██████████| 1245/1245 [06:34<00:00,  3.16it/s]

✅ Completed inference on 1245 samples
📄 Results saved to results.jsonl

📊 Sample results:
1. Q: What color is Wang Hua's car?
   Pred: blue
   GT: Blue

2. Q: Is the animal in the chair one of the zodiac signs?
   Pred: No
   GT: Yes

3. Q: Is the man in the black suit the 44th president?
   Pred: Yes
   GT: Yes

4. Q: Where is the building in the picture located?
   Pred: London
   GT: The British Museum in London, England

5. Q: What is the name of the person in the picture?
   Pred: Laura
   GT: Laura



In [2]:
import collections
import json

def maf1(pred, gt):
    """Word-level F1"""
    pred_tokens = pred.split()
    gt_tokens = gt.split()

    common = collections.Counter(gt_tokens) & collections.Counter(pred_tokens)
    num_same = sum(common.values())

    if len(gt_tokens) == 0 or len(pred_tokens) == 0:
        return int(gt_tokens == pred_tokens)
    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def vtqa_eval_jsonl(pred_jsonl, yn_answer):
    pred_data = []
    with open(pred_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            pred_data.append(json.loads(line.strip()))

    em_right = yn_right = yn_em = yn_num = 0
    extract_f1 = e_em = extract_num = 0
    generate_f1 = g_em = generate_num = 0

    for p in pred_data:
        pred = p['pred'].lower().strip()
        gt = p['gt'].lower().strip()
        ans_type = p['answer_type']

        # Exact match
        if pred == gt:
            em_right += 1

        if ans_type == 'YN':
            yn_num += 1
            if pred == gt:
                yn_em += 1
            if pred in yn_answer['yes'] and gt == 'yes':
                yn_right += 1
            elif pred in yn_answer['no'] and gt == 'no':
                yn_right += 1

        elif ans_type == 'E':  # Extractive
            if pred == gt:
                e_em += 1
            extract_num += 1
            extract_f1 += maf1(pred, gt)

        elif ans_type == 'G':  # Generative
            if pred == gt:
                g_em += 1
            generate_num += 1
            generate_f1 += maf1(pred, gt)

    # Metrics
    em_all_acc = em_right / len(pred_data)
    print(f'all answer num: {len(pred_data)}, exact match acc: {em_all_acc:.4f}')

    yn_em_acc = yn_em / yn_num if yn_num != 0 else 0
    yn_acc = yn_right / yn_num if yn_num != 0 else 0
    print(f'YN answer num: {yn_num}, YN acc: {yn_acc:.4f}, YN EM: {yn_em_acc:.4f}')

    e_em_acc = e_em / extract_num if extract_num != 0 else 0
    e_f1 = extract_f1 / extract_num if extract_num != 0 else 0
    print(f'Extract answer num: {extract_num}, E F1: {e_f1:.4f}, E EM: {e_em_acc:.4f}')

    g_em_acc = g_em / generate_num if generate_num != 0 else 0
    g_f1 = generate_f1 / generate_num if generate_num != 0 else 0
    print(f'Generate answer num: {generate_num}, G F1: {g_f1:.4f}, G EM: {g_em_acc:.4f}')

    return em_all_acc, yn_acc, yn_em_acc, e_f1, e_em_acc, g_f1, g_em_acc


In [3]:
yn_answer = {
    "yes": ["yes", "yeah", "yep"],
    "no": ["no", "nah", "nope"]
}

vtqa_eval_jsonl("results.jsonl", yn_answer)


all answer num: 1245, exact match acc: 0.3896
YN answer num: 436, YN acc: 0.7156, YN EM: 0.7156
Extract answer num: 763, E F1: 0.3704, E EM: 0.2123
Generate answer num: 46, G F1: 0.2996, G EM: 0.2391


(0.3895582329317269,
 0.7155963302752294,
 0.7155963302752294,
 0.3704007627285253,
 0.21231979030144169,
 0.29963768115942024,
 0.2391304347826087)

In [ ]:
all answer num: 1245, exact match acc: 0.2843
YN answer num: 436, YN acc: 0.5963, YN EM: 0.5963
Extract answer num: 763, E F1: 0.2315, E EM: 0.1219
Generate answer num: 46, G F1: 0.1226, G EM: 0.0217

In [ ]:
0.28433734939759037,
 0.5963302752293578,
 0.5963302752293578,
 0.23149787855751525,
 0.1218872870249017,
 0.12573392790784096,
 0.043478260869565216)

In [ ]:
import collections
import json

def maf1(pred, gt):
    """Word-level F1"""
    pred_tokens = pred.split()
    gt_tokens = gt.split()

    common = collections.Counter(gt_tokens) & collections.Counter(pred_tokens)
    num_same = sum(common.values())

    if len(gt_tokens) == 0 or len(pred_tokens) == 0:
        return int(gt_tokens == pred_tokens)
    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def vtqa_eval_jsonl(pred_jsonl, yn_answer):
    pred_data = []
    with open(pred_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            pred_data.append(json.loads(line.strip()))

    em_right = yn_right = yn_em = yn_num = 0
    extract_f1 = e_em = extract_num = 0
    generate_f1 = g_em = generate_num = 0

    for p in pred_data:
        pred = p['pred'].lower().strip()
        gt = p['gt'].lower().strip()
        ans_type = p['answer_type']

        # Exact match cho toàn bộ dataset
        if pred == gt:
            em_right += 1

        if ans_type == 'YN':
            yn_num += 1
            if pred == gt:
                yn_em += 1
            if pred in yn_answer['yes'] and gt == 'yes':
                yn_right += 1
            elif pred in yn_answer['no'] and gt == 'no':
                yn_right += 1

        elif ans_type == 'E':  # Extractive
            extract_num += 1
            if pred == gt:
                e_em += 1
            extract_f1 += maf1(pred, gt)

        elif ans_type == 'G':  # Generative
            generate_num += 1
            if pred == gt:
                g_em += 1
                generate_f1 += 1
            elif gt in pred:  # containment → coi như match
                g_em += 1
                generate_f1 += 1
            else:
                generate_f1 += maf1(pred, gt)

    # Metrics
    em_all_acc = em_right / len(pred_data)
    print(f'all answer num: {len(pred_data)}, exact match acc: {em_all_acc:.4f}')

    yn_em_acc = yn_em / yn_num if yn_num != 0 else 0
    yn_acc = yn_right / yn_num if yn_num != 0 else 0
    print(f'YN answer num: {yn_num}, YN acc: {yn_acc:.4f}, YN EM: {yn_em_acc:.4f}')

    e_em_acc = e_em / extract_num if extract_num != 0 else 0
    e_f1 = extract_f1 / extract_num if extract_num != 0 else 0
    print(f'Extract answer num: {extract_num}, E F1: {e_f1:.4f}, E EM: {e_em_acc:.4f}')

    g_em_acc = g_em / generate_num if generate_num != 0 else 0
    g_f1 = generate_f1 / generate_num if generate_num != 0 else 0
    print(f'Generate answer num: {generate_num}, G F1: {g_f1:.4f}, G EM: {g_em_acc:.4f}')

    return em_all_acc, yn_acc, yn_em_acc, e_f1, e_em_acc, g_f1, g_em_acc


In [ ]:
yn_answer = {
    "yes": ["yes", "yeah", "yep"],
    "no": ["no", "nah", "nope"]
}

vtqa_eval_jsonl("results.jsonl", yn_answer)
